## Добрый день. 
Это вторая часть задания по создания линейных классификаторов.

В примере будет использована библиотека pytorch. 
В качестве модели выступает, как и в первой части, логистическая регрессия.   
В основе работы лежит решение задачи многоклассовой классификации. Градиент ошибки вычислияется с помощью cross_entropy. 
Классификатор использует архитектуру нейронной сети с одним линейным слоем. Обратное распространение ошибки и обновление весов осуществляются в автоматическом режиме.

### Загрузка данных и библиотек

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time

import pandas as pd
from sklearn.metrics import f1_score, classification_report

Создаем возможность стабильных репродуктивных результатов рерана тетрадки.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Для нормализации изображений рассчитаем среднее и среднеквадратичное отклонение для каждого из трех цветновых слоев.

In [3]:
ROOT = r'F:\\temp_pics\\cifar'

train_data = datasets.CIFAR10(root = ROOT, 
                              train = True, 
                              download = False)

means = train_data.data.mean(axis = (0,1,2)) / 255
stds = train_data.data.std(axis = (0,1,2)) / 255

print(f'Средние значения: {means}')
print(f'Отклонения: {stds}')

Средние значения: [0.49139968 0.48215841 0.44653091]
Отклонения: [0.24703223 0.24348513 0.26158784]


### Подготовка датасета

Создаем трансформер, который преобразует исходные изображения для дальнейшей работы модели. Процесс изменения исходных фоторгафий называется аугментация.  
В тестовую выборку добавим случайный поворот изображений на 5 градусов, отражение изображий по горизонтали, случайный кроп изображений. Аугментация поможет модели обучиться на большем разнообразии данных без добавления в датасет новых реальных фотографий.
После изменений изображений, трансформер преобразует массивы данных в тензоры и нормализует их для подачи на вход модели.  

Аугментация проводится только на обучающих данных. Искажение тестовых данных может испортить валидацию модели.
  
*При реране ноутбука на локальной машине - пожалуйста, скачайте архив [cifar_10](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz) в любую папку. Далее - измените путь к данным в ячейке ниже.*

In [4]:
train_transforms = transforms.Compose([
                           transforms.RandomRotation(5),
                           transforms.RandomHorizontalFlip(0.5),
                           transforms.RandomCrop(32, padding = 2),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = means, 
                                                std = stds)
                       ])

test_transforms = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean = means, 
                                                std = stds)
                       ])

Загружаем данные и прогоняем через описанные выше трансформеры.

In [5]:
train_data = datasets.CIFAR10(ROOT, 
                              train = True, 
                              download = False, 
                              transform = train_transforms)

test_data = datasets.CIFAR10(ROOT, 
                             train = False, 
                             download = False, 
                             transform = test_transforms)

Для вариативности проверки результатов работы модели создадим валидационный датасет.  
Применяем к нему трансформацию тестового датасета. Команда deepcopy() позволит избежать изменений в структуре valid_data, которая не подвергалась изменениям.

In [6]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])  

valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

Исходные данные разделены на 3 сета.

In [7]:
print(f'Тренировочный датасет: {len(train_data)}')
print(f'Валидационный сет: {len(valid_data)}')
print(f'Тестовый сет: {len(test_data)}')

Тренировочный датасет: 45000
Валидационный сет: 5000
Тестовый сет: 10000


Для обучения модели необходимо создать итератор батчей. Батчи - небольшие наборы данных, несут две важные функции. Во-первых, обучение по частям позволяет занимать в памяти вычислительной машины меньше места, чем целый набор данных. Во-вторых, обучение на батчах позволяет усреднять ошибку и предсказания модели для различных классов. Так мы получаем более стабильный и качественный результат.  

В данном примере я использую размер батча = 64 изображениям. Можно брать бОльший размер, который увеличит скорость обучения. Но текущая конфигурация локальной машины позволяет мне комфортно работать именно с таким размером выборки.

In [8]:
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

test_iterator = data.DataLoader(test_data, 
                                batch_size = BATCH_SIZE)

### Создание модели  

Цель задания - написать любой линейный классификатор на pytorch. Предлагаю создать, как и в случае с numpy, логистическую регрессию и посмотреть, какой результат нам покажет эта модель.  

Строим нейросеть, как бы серьезно это не звучало, с одним линейным слоем. В данном примере не используются свертки и сложные предобученные архитектуры. Интересно именно сравнить простые решения.

In [9]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.linear = torch.nn.Linear(3*32*32, output_dim)

    def forward(self, x):
        # этот шаг необходим для перевода трехмерного изображения о двумерный массив для подачи на линейный слой
        x = x.view(x.shape[0], -1) 
        # уже обычный подсчет весов логистической регрессии
        outputs = self.linear(x)
        return outputs

Из гиперпараметров передадим модели только размер выходного тензора. Решая задачу мультиклассовой классификации мы находим принадлежность изображения к одному из них. Тензор длинной 10 будет содержать вероятностное значение для каждого из десяти классов - какое изображение перед нами.

In [10]:
OUTPUT_DIM = 10

classifier_pytorch = LogisticRegression(OUTPUT_DIM)

Посмотрим на количество параметров нашей логистической регрессии.

In [11]:
def count_parameters(classifier_pytorch):
    return sum(p.numel() for p in classifier_pytorch.parameters() if p.requires_grad)

print(f'The model has {count_parameters(classifier_pytorch):,} trainable parameters')

The model has 30,730 trainable parameters


В отличии от простых нейросетей с несколькими слоями и миллионом параметров в базе, наша малютка имеет всего лишь 31 тысячу параметров.   

В качестве функции оптимизации исользуем алгоритм Adam. За функцию потерь, как и в случае с классификатором на numpy, отвечает cross-entropy. Единственное отличие - в pytorch их не надо прописывать вручную.

In [12]:
optimizer = optim.Adam(classifier_pytorch.parameters())
criterion = nn.CrossEntropyLoss()

Проверяем возможность обучения модели на GPU и передаем в память видеокарты модель и функцию потерь.

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classifier_pytorch = classifier_pytorch.to(device)
criterion = criterion.to(device)

In [14]:
device

device(type='cuda')

Создадим функцию подсчета accuracy

In [15]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

Создаем функции обучения и тестирования модели.  

Переводим модель в состояние train().  
Выбираем изображения для батча.  
Обнуляем градиент предыдущего батча.
Получаем предикты модели на данных батча.  
Рассчитываем величину ошибки между предиктами и реальными значениями.  
Смотрим на accuracy между предиктами и реальными лейблами.  
Рассчитываем градиенты параметров.  
Обновлаяем параметры командой step().  
Обновляем метрики loss эпохи и accuracy эпохи.

In [16]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
      
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Для проверки работы модели создаем функцию evaluate()

Переводим модель в состояние eval()  
Функция torch.no_grad() отключает вычисления градиента и ошибки. На этом этапе нам не нужно совершать обратные преобразования.  
Увеличиваем скорость работы и снижаем использование памяти.

In [17]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Создаем функию подсчета времени эпохи обучения

In [18]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Обучаем логистическую регрессию на пяти эпохах. В черновых версиях работы я довел обучение до 50 эпох, улучшения метрик нет. 

In [19]:
EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.monotonic()
    
    train_loss, train_acc = train(classifier_pytorch, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(classifier_pytorch, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        # чтобы сохранить обученную модель с весами, раскомментируйте строку ниже и укажите путь сохранения.
        #torch.save(model.state_dict(), 'path\\awesome_logistic_regression.pt')
    
    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 51s
	Train Loss: 2.079 | Train Acc: 29.95%
	 Val. Loss: 2.059 |  Val. Acc: 32.54%
Epoch: 02 | Epoch Time: 0m 32s
	Train Loss: 2.063 | Train Acc: 31.27%
	 Val. Loss: 1.978 |  Val. Acc: 34.93%
Epoch: 03 | Epoch Time: 0m 32s
	Train Loss: 2.045 | Train Acc: 32.02%
	 Val. Loss: 2.000 |  Val. Acc: 33.43%
Epoch: 04 | Epoch Time: 0m 34s
	Train Loss: 2.047 | Train Acc: 32.07%
	 Val. Loss: 2.030 |  Val. Acc: 35.21%
Epoch: 05 | Epoch Time: 0m 32s
	Train Loss: 2.050 | Train Acc: 32.10%
	 Val. Loss: 2.082 |  Val. Acc: 33.03%


Проверим работу модели на тестовых данных

In [20]:
test_loss, test_acc = evaluate(classifier_pytorch, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 2.068 | Test Acc: 32.63%


Невысокая точность предсказаний. Результаты близки к значениям sklearn и классификатору numpy.

### Проверка работы модели  

Создадим функцию для предсказания массивов изображений.

In [21]:
def get_predictions(model, iterator, device):

    model.eval()

    labels = []
    probs = []

    with torch.no_grad():

        for (x, y) in iterator:

            x = x.to(device)

            y_pred = model(x)

            y_prob = F.softmax(y_pred, dim = -1)
            top_pred = y_prob.argmax(1, keepdim = True)

            labels.append(y.cpu())
            probs.append(y_prob.cpu())

    labels = torch.cat(labels, dim = 0)
    probs = torch.cat(probs, dim = 0)
    pred_labels = torch.argmax(probs, 1)
    
    return labels, pred_labels

In [22]:
true, preds = get_predictions(classifier_pytorch, test_iterator, device)

In [23]:
f1_score(true, preds, average='macro')

0.3117884366830577

Проверим f1 score на предиктах модели.

In [24]:
def evaluation(data):
    
    true, preds = get_predictions(classifier_pytorch, data, device)
    score = f1_score(true, preds, average='macro')
    class_report = classification_report(true, preds)
    
    print('f1-macro скор модели =', np.round(score, 4))
    print()
    print(class_report)

In [25]:
evaluation(train_iterator)

f1-macro скор модели = 0.3004

              precision    recall  f1-score   support

           0       0.30      0.24      0.26      4501
           1       0.36      0.46      0.41      4495
           2       0.21      0.28      0.24      4485
           3       0.23      0.22      0.23      4518
           4       0.29      0.22      0.25      4458
           5       0.23      0.18      0.20      4533
           6       0.35      0.26      0.30      4536
           7       0.33      0.38      0.35      4502
           8       0.37      0.46      0.41      4485
           9       0.37      0.34      0.35      4487

    accuracy                           0.30     45000
   macro avg       0.30      0.30      0.30     45000
weighted avg       0.30      0.30      0.30     45000



In [26]:
evaluation(valid_iterator)

f1-macro скор модели = 0.315

              precision    recall  f1-score   support

           0       0.35      0.22      0.27       499
           1       0.41      0.43      0.42       505
           2       0.22      0.25      0.23       515
           3       0.24      0.16      0.19       482
           4       0.35      0.27      0.31       542
           5       0.34      0.14      0.20       467
           6       0.36      0.31      0.33       464
           7       0.37      0.41      0.39       498
           8       0.33      0.57      0.42       515
           9       0.33      0.50      0.40       513

    accuracy                           0.33      5000
   macro avg       0.33      0.33      0.31      5000
weighted avg       0.33      0.33      0.32      5000



In [27]:
evaluation(test_iterator)

f1-macro скор модели = 0.3118

              precision    recall  f1-score   support

           0       0.30      0.19      0.23      1000
           1       0.43      0.43      0.43      1000
           2       0.22      0.26      0.24      1000
           3       0.27      0.17      0.21      1000
           4       0.29      0.27      0.28      1000
           5       0.29      0.12      0.17      1000
           6       0.39      0.30      0.34      1000
           7       0.36      0.42      0.39      1000
           8       0.32      0.59      0.42      1000
           9       0.35      0.52      0.42      1000

    accuracy                           0.33     10000
   macro avg       0.32      0.33      0.31     10000
weighted avg       0.32      0.33      0.31     10000



Обновим табличку из первого задания.

In [30]:
models = ['numpy', 'sklearn', 'pytorch']
time = [15.7, 10.1, 118.]
train_score = [0.446, 0.582, 0.299]
valid_score = [0.311, 0.311, 0.315]
test_score = [0.300, 0.315, 0.312]

dict_ = {'algorithm':models,
         'training time':time, 
         'f1_train':train_score,
         'f1_valid':valid_score,
         'f1_test':test_score
        }

result = pd.DataFrame(data=dict_)

In [31]:
result

,algorithm,training time,f1_train,f1_valid,f1_test
0,numpy,15.7,0.446,0.311,0.300
1,sklearn,10.1,0.582,0.311,0.315
2,pytorch,118.0,0.299,0.315,0.312


В модели нет переобучения! Однако заметна слабая недообученность. 
Валидационную и тестовую выборки модель предсказывает на уровне numpy и коробочного решения. Однако, очень сильно проигрывает по времени.  
Анализ classification report показал, что модель лучше угадывает те же классы, что и решения из предыдущей работы.

### Вывод  

В данной тетради представлена работа простой мультиклассовой логистической регрессии, написанной на pytorch. Обучающие и тестовые данные являются частью набора Cifar10.   
Градиент ошибки рассчитывается с помощью softmax и cross-entropy.  

Написанная "нейросеть" содержит 1 линейный слой. На выходе модели получаем числовые значения классов. Прогоняем их через softmax функцию для расчета вероятности принадлежности к классу.  

Модель показала значения качества предсказий близкие к модели с архитектурой на numpy и решением sklearn. Единственное, в чем сильно проигрывает получанный классификатор - время обучения и предсказания.  

Улучшить качество модели можно изменением архитектуры и использовании моделей с предобученными параметрами.